In [82]:
# Import the required modules
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as T
from torchvision.datasets import MNIST
from torch.utils.data import random_split, DataLoader
from tqdm import tqdm
import matplotlib.pyplot as plt
import numpy as np

In [8]:
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

In [130]:
HP = {
    "lr": [1e-1,1e-2,1e-3,1e-4],
    "hidden_layers": [[200, 100, 50], [200, 100], [200]],
    "activation_funcs": [nn.ReLU(),nn.LeakyReLU(),nn.Sigmoid()],
    "batch_size":64,
    "num_epoch":3
}

In [10]:
from torchvision.datasets import CIFAR10
import torchvision.transforms as T

train_transform = T.Compose ([
# can add additional transforms on images
T.ToTensor(), # convert images to PyTorch tensors
T.Grayscale(), # RGB to grayscale
T.Normalize( mean =(0.5 ,) , std=(0.5 ,) ) # n or ma li za ti on
# speeds up the convergence
# and improves the accuracy
])
val_transform = test_transform = T.Compose([
T.ToTensor() ,
T.Grayscale() ,
T.Normalize( mean =(0.5 ,) , std=(0.5 ,) )
])

train_set = CIFAR10 ( root ="CIFAR10", train =True ,transform = train_transform , download = True )
test_set = CIFAR10 ( root ="CIFAR10", train =False ,transform = test_transform , download = True )
train_set_length = int(0.8 * len(train_set))
val_set_length = len(train_set) - train_set_length
train_set, val_set = random_split(train_set, [train_set_length, val_set_length])


train_loader = DataLoader(train_set, batch_size=HP["batch_size"], shuffle=True)
test_loader = DataLoader(test_set, batch_size=HP["batch_size"])
val_loader = DataLoader(val_set, batch_size=HP["batch_size"])


Files already downloaded and verified
Files already downloaded and verified


In [11]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [134]:
# Define the ANN
class MyModel(nn.Module):

    def __init__(self, output_shape, hidden_layers,activation):
        super().__init__()
        layers = [32*32*1] + hidden_layers + [output_shape]
        modules = []
        
        for i in range(len(layers)-1):
            modules.append(nn.Linear(layers[i], layers[i+1]))
            modules.append(activation)
        self.lin = nn.Sequential(*modules)

    def forward(self, x):
        return self.lin(x)

In [135]:
for i in range(len(HP["hidden_layers"])):
    for j in range(len(HP["lr"])):
        for k in range(len(HP["activation_funcs"])):

            model = MyModel(10, hidden_layers=(HP["hidden_layers"][i]),activation=HP["activation_funcs"][k]).to(device)
            loss_function = nn.CrossEntropyLoss()
            optimizer = torch.optim.Adam(model.parameters(), lr=HP["lr"][j])
            
 
            for epoch in tqdm(range(HP["num_epoch"])):
                # Training
                model.train()
                accum_train_loss = 0
                for i, (imgs, labels) in enumerate(train_loader, start=1):
                    tqdm()
                    imgs, labels = imgs.to(device), labels.to(device)
                    output = model(imgs)
                    loss = loss_function(output, labels)

                    # accumlate the loss
                    accum_train_loss += loss.item()

                    # backpropagation
                    optimizer.zero_grad()
                    loss.backward()
                    optimizer.step()
                
                # Validation
                model.eval()
                accum_val_loss = 0
                with torch.no_grad():
                    for j, (imgs, labels) in enumerate(val_loader, start=1):
                        imgs, labels = imgs.to(device), labels.to(device)
                        output = model(imgs)
                        accum_val_loss += loss_function(output, labels).item()

                # print statistics of the epoch
                print(f'Epoch = {epoch} | Train Loss = {accum_train_loss / i:.4f}\tVal Loss = {accum_val_loss / j:.4f}')

---------------training with---------------
0.1
ReLU()
MyModel(
  (lin): Sequential(
    (0): Linear(in_features=1024, out_features=200, bias=True)
    (1): ReLU()
    (2): Linear(in_features=200, out_features=100, bias=True)
    (3): ReLU()
    (4): Linear(in_features=100, out_features=50, bias=True)
    (5): ReLU()
    (6): Linear(in_features=50, out_features=10, bias=True)
    (7): ReLU()
  )
)
---------------training with---------------
0.1
LeakyReLU(negative_slope=0.01)
MyModel(
  (lin): Sequential(
    (0): Linear(in_features=1024, out_features=200, bias=True)
    (1): LeakyReLU(negative_slope=0.01)
    (2): Linear(in_features=200, out_features=100, bias=True)
    (3): LeakyReLU(negative_slope=0.01)
    (4): Linear(in_features=100, out_features=50, bias=True)
    (5): LeakyReLU(negative_slope=0.01)
    (6): Linear(in_features=50, out_features=10, bias=True)
    (7): LeakyReLU(negative_slope=0.01)
  )
)
---------------training with---------------
0.1
Sigmoid()
MyModel(
  (lin): Se

In [ ]:
for epoch in tqdm(range(num_epoch)):
    # Training
    model.train()
    accum_train_loss = 0
    for i, (imgs, labels) in enumerate(train_loader, start=1):
        tqdm()
        imgs, labels = imgs.to(device), labels.to(device)
        output = model(imgs)
        loss = loss_function(output, labels)

        # accumlate the loss
        accum_train_loss += loss.item()

        # backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    # Validation
    model.eval()
    accum_val_loss = 0
    with torch.no_grad():
        for j, (imgs, labels) in enumerate(val_loader, start=1):
            imgs, labels = imgs.to(device), labels.to(device)
            output = model(imgs)
            accum_val_loss += loss_function(output, labels).item()

    # print statistics of the epoch
    print(f'Epoch = {epoch} | Train Loss = {accum_train_loss / i:.4f}\tVal Loss = {accum_val_loss / j:.4f}')

In [ ]:

print("---------------training with---------------")
print(HP["lr"][j])
print(HP["activation_funcs"][k])
print(model)